In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/srifoton-25-machine-learning-competition/val.csv
/kaggle/input/srifoton-25-machine-learning-competition/train.csv
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0398.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0063.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0061.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0178.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0064.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0378.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0328.jpeg
/kaggle/input/srifoton-25-machine-learning-competition/val/val/Tuberculosis/val_tuberculosis_0321.jpeg
/kaggle/input/srifoton-25-machine-learning-compe

In [2]:
# =============================================================================
# 1. SETUP DAN IMPOR LIBRARY
# =============================================================================
import numpy as np
import pandas as pd
import os
from PIL import Image
import gc # Garbage Collector

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, datasets
from torch.optim.lr_scheduler import CosineAnnealingLR # Scheduler yang lebih baik
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

print("Semua library berhasil diimpor!")

# =============================================================================
# 2. KONFIGURASI DAN DATA
# =============================================================================
# --- Konfigurasi ---
BASE_PATH = '/kaggle/input/srifoton-25-machine-learning-competition/'
TRAIN_VAL_DIR = os.path.join(BASE_PATH, 'train/train') # Kita akan gabungkan train & val
TEST_DIR = os.path.join(BASE_PATH, 'test/test')

# --- Hyperparameter ---
IMG_SIZE = 256
BATCH_SIZE = 32
LEARNING_RATE = 1e-4 # Coba LR yang lebih konservatif
NUM_EPOCHS = 15    # Jumlah epoch per fold
NUM_CLASSES = 5
N_SPLITS = 5       # Jumlah folds untuk Cross-Validation

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Menggunakan device: {DEVICE}")

# --- Transformasi yang lebih aman ---
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- Custom Dataset ---
class XRayDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(label, dtype=torch.long)

# =============================================================================
# 3. PROSES TRAINING DENGAN K-FOLD CROSS-VALIDATION
# =============================================================================
# Muat semua data training dan gabungkan path dengan labelnya
full_dataset_info = datasets.ImageFolder(TRAIN_VAL_DIR)
all_image_paths = np.array([p for p, l in full_dataset_info.samples])
all_labels = np.array(full_dataset_info.targets)

# Inisialisasi StratifiedKFold
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Loop utama untuk Cross-Validation
for fold, (train_idx, val_idx) in enumerate(skf.split(all_image_paths, all_labels)):
    print(f"\n{'='*20} FOLD {fold+1}/{N_SPLITS} {'='*20}")

    # 1. Definisikan data untuk fold ini
    train_paths, val_paths = all_image_paths[train_idx], all_image_paths[val_idx]
    train_labels, val_labels = all_labels[train_idx], all_labels[val_idx]

    train_dataset = XRayDataset(train_paths, train_labels, transform=train_transforms)
    val_dataset = XRayDataset(val_paths, val_labels, transform=val_test_transforms)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

    # 2. Inisialisasi ulang model, optimizer, dan scheduler di setiap fold
    model = models.efficientnet_b2(weights='IMAGENET1K_V1') # Coba B2 untuk kapasitas lebih
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, NUM_CLASSES)
    model = model.to(DEVICE)

    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # Scheduler yang lebih modern: CosineAnnealingLR
    scheduler = CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS, eta_min=1e-6)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    
    # 3. Training Loop untuk fold ini
    best_f1 = 0.0
    for epoch in range(NUM_EPOCHS):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)

        model.eval()
        all_preds, all_labels_val = [], []
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels_val.extend(labels.cpu().numpy())

        macro_f1 = f1_score(all_labels_val, all_preds, average='macro')
        print(f"Epoch {epoch+1}/{NUM_EPOCHS} | Loss: {running_loss/len(train_dataset):.4f} | Val Macro F1: {macro_f1:.4f}")

        if macro_f1 > best_f1:
            best_f1 = macro_f1
            torch.save(model.state_dict(), f'best_model_fold_{fold}.pth')
            print(f"-> Model Fold {fold} disimpan dengan F1: {best_f1:.4f}")
        
        scheduler.step() # Update scheduler

    # Membersihkan memori GPU
    del model, train_loader, val_loader
    gc.collect()
    torch.cuda.empty_cache()




Semua library berhasil diimpor!
Menggunakan device: cuda

==================== FOLD 1/5 ====================


Downloading: "https://download.pytorch.org/models/efficientnet_b2_rwightman-c35c1473.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b2_rwightman-c35c1473.pth
100%|██████████| 35.2M/35.2M [00:00<00:00, 162MB/s] 


Epoch 1/15 | Loss: 0.8602 | Val Macro F1: 0.8727
-> Model Fold 0 disimpan dengan F1: 0.8727
Epoch 2/15 | Loss: 0.6283 | Val Macro F1: 0.9085
-> Model Fold 0 disimpan dengan F1: 0.9085
Epoch 3/15 | Loss: 0.5743 | Val Macro F1: 0.9111
-> Model Fold 0 disimpan dengan F1: 0.9111
Epoch 4/15 | Loss: 0.5541 | Val Macro F1: 0.9098
Epoch 5/15 | Loss: 0.5268 | Val Macro F1: 0.9086
Epoch 6/15 | Loss: 0.5020 | Val Macro F1: 0.9164
-> Model Fold 0 disimpan dengan F1: 0.9164
Epoch 7/15 | Loss: 0.4817 | Val Macro F1: 0.9181
-> Model Fold 0 disimpan dengan F1: 0.9181
Epoch 8/15 | Loss: 0.4708 | Val Macro F1: 0.9133
Epoch 9/15 | Loss: 0.4555 | Val Macro F1: 0.9131
Epoch 10/15 | Loss: 0.4539 | Val Macro F1: 0.9169
Epoch 11/15 | Loss: 0.4434 | Val Macro F1: 0.9235
-> Model Fold 0 disimpan dengan F1: 0.9235
Epoch 12/15 | Loss: 0.4430 | Val Macro F1: 0.9224
Epoch 13/15 | Loss: 0.4326 | Val Macro F1: 0.9235
-> Model Fold 0 disimpan dengan F1: 0.9235
Epoch 14/15 | Loss: 0.4325 | Val Macro F1: 0.9216
Epoch 15

In [9]:
# =============================================================================
# 4. PREDIKSI ENSEMBLE DARI SEMUA MODEL FOLD
# =============================================================================
print(f"\nMemulai prediksi ensemble dari {N_SPLITS} model...")

# Dataset untuk test
class TestDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.image_paths = sorted(os.listdir(root_dir))
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, self.image_paths[idx]

test_dataset = TestDataset(TEST_DIR, transform=val_test_transforms)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Kumpulkan probabilitas dari semua model
all_fold_probs = []
for fold in range(N_SPLITS):
    model = models.efficientnet_b2(weights='IMAGENET1K_V1')
    in_features = model.classifier[1].in_features
    model.classifier[1] = nn.Linear(in_features, NUM_CLASSES)
    model.load_state_dict(torch.load(f'best_model_fold_{fold}.pth'))
    model = model.to(DEVICE)
    model.eval()

    fold_probs = []
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(DEVICE)
            outputs = model(inputs)
            probs = torch.nn.functional.softmax(outputs, dim=1).cpu().numpy()
            fold_probs.append(probs)
    all_fold_probs.append(np.concatenate(fold_probs))

# Rata-ratakan probabilitas dan ambil prediksi akhir
avg_probs = np.mean(all_fold_probs, axis=0)
final_preds = np.argmax(avg_probs, axis=1)

# Buat file submisi
submission_df = pd.DataFrame({'Id': test_dataset.image_paths, 'Predicted': final_preds})
submission_df.to_csv('submission_cv.csv', index=False)

print("\nFile submission_cv.csv berhasil dibuat!")
print(submission_df.head())


Memulai prediksi ensemble dari 5 model...

File submission_cv.csv berhasil dibuat!
               Id  Predicted
0  test_0001.jpeg          3
1  test_0002.jpeg          0
2  test_0003.jpeg          2
3  test_0004.jpeg          4
4  test_0005.jpeg          2


In [11]:
from IPython.display import FileLink
FileLink('submission_cv.csv')

/kaggle/working/submission_cv.csv